In [ ]:
# http://www.vgchartz.com/gamedb/
# https://github.com/GregorUT/vgchartzScrape

In [ ]:
from bs4 import BeautifulSoup, element
from urllib.request import urlopen
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
# https://medium.com/@harshit4084/track-your-loop-using-tqdm-7-ways-progress-bars-in-python-make-things-easier-fcbbb9233f24

In [ ]:
import requests
from time import sleep
import re

In [ ]:
url_base = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?sort=desc&view=condensed&page='

In [ ]:
g_name = []
g_platform = []
critic_score = []
user_score = []
g_publisher = []
g_genre = []
g_release_date = []

for p in tqdm(range(0,192), desc= 'Main loop metacritic'):  #0 192
    url = url_base + str(p)
      
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')    
   
    href_ = soup.findAll(name = 'a', attrs = {'class': 'title'})
  
    for elt in tqdm(href_, total=len(href_), desc= f'page {p}'):
        url_to_game = 'https://www.metacritic.com'+elt.attrs['href']
        #print (url_to_game)
        response2 = requests.get(url_to_game, headers = user_agent)
        sub_soup = BeautifulSoup(response2.text, 'html.parser')
        
        info = sub_soup.findAll(name = 'a', attrs = {'class': 'hover_none'})
        try:
            g_name.append(info[0].text.strip())
        except:
            continue
        
        info = sub_soup.findAll(name = 'span', attrs = {'class': 'platform'})
        g_platform.append(info[0].text.strip())

        info = sub_soup.findAll(name = 'span', attrs = {'itemprop': 'ratingValue'})
        try:
            critic_score.append(float(info[0].text.strip()))
        except:
            critic_score.append(np.nan)

        info = sub_soup.find_all("div", class_=re.compile("^metascore_w user large game"))
        try:
            user_score.append(float(info[0].text.strip()))
        except:
            user_score.append(np.nan)
            
        info = sub_soup.select(".summary_details a")
        try:
            g_publisher.append(info[0].text.strip())
        except:
            g_publisher.append(np.nan)
        
        info = sub_soup.findAll(name = 'li', attrs = {'class': 'summary_detail release_data'})           
        rel_date = info[0].findAll('span', attrs = {'class': 'data'})
        g_release_date.append(rel_date[0].text.strip())
        
        info = sub_soup.findAll(name = 'li', attrs = {'class': 'summary_detail product_genre'})           
        #t = info[0].text.strip()
        genre = info[0].findAll('span', attrs = {'class': 'data'})
        genre = [g.text.strip() for g in genre]
        g_genre.append(','.join(genre))
        sleep(0.5)

metacritic = pd.DataFrame({'title': g_name,
              'platform': g_platform,
              'publisher': g_publisher,
          #    'developer': g_developer,
              'critic_score': critic_score,
              'user_score': user_score,
              'release date' : g_release_date,
              'genre': g_genre,
             })

metacritic.head()



In [ ]:
metacritic.info()

In [ ]:
import matplotlib.pyplot as plt
metacritic.critic_score.min(), metacritic.user_score.min(), metacritic.user_score.max()

In [ ]:
plt.scatter(metacritic.critic_score, metacritic.user_score)

In [ ]:
metacritic.to_csv("_vgsales_metacritic_v2.csv", sep=",", encoding='utf-8', index=False)